In [10]:
pip install pysindy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import numpy as np
import pandas as pd
import random
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pysindy as ps
from sklearn.preprocessing import StandardScaler
from dynamics_generator_3d import SIR_model,Linear_Dynamics_3D,Lorenz_System,BZ_System
from dynamics_generator_3d import generate_Linear,generate_SIR,generate_Lorenz,generate_BZ

In [12]:
Linear_generated = generate_Linear(N = 1000, noise = None)
SIR_generated = generate_SIR(N = 1000, noise = None)
Lorenz_generated = generate_Lorenz(N = 1000, noise = None)
BZ_generated = generate_BZ(N = 1000, noise = None)
Linear_generated_G = generate_Linear(N = 1000, noise = 'Gaussian')
SIR_generated_G = generate_SIR(N = 1000, noise = 'Gaussian')
Lorenz_generated_G = generate_Lorenz(N = 1000, noise = 'Gaussian')
BZ_generated_G = generate_BZ(N = 1000, noise = 'Gaussian')

In [13]:
Data = np.concatenate([Linear_generated, SIR_generated, Lorenz_generated,BZ_generated,
                       Linear_generated_G, SIR_generated_G, Lorenz_generated_G, BZ_generated_G], axis = 0)
np.random.shuffle(Data)

In [5]:
X_data, y_data = [], []
for series, time, label in Data:
  try:
    model_sindy = ps.SINDy()
    model_sindy.fit(series, t = time)
    coef = model_sindy.coefficients().reshape(-1)
    X_data.append(coef)
    y_data.append(label)
  except:
    pass


/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coefficients
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysindy/optimizers/stlsq.py:201: UserWarning: Sparsity parameter is too big (0.1) and eliminated all coef

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, stratify = y_data, test_size = 0.1, random_state = 42)

#2. Prediction

In [42]:
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.linear_model import LogisticRegression

## 2.1 SVM

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, stratify = y_data, test_size = 0.1, random_state = 42)

### 2.1.1 Linear SVM

In [35]:
from sklearn.svm import LinearSVC, SVC

lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train,y_train)

from sklearn.metrics import accuracy_score
y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.71625

In [36]:
from sklearn.metrics import confusion_matrix    
y_pred = lin_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))                    
print(confusion_matrix(y_test, y_pred) )

0.6975
[[ 48 103  42   7]
 [ 14 166   9  11]
 [ 17  19 163   1]
 [  8   5   6 181]]


### 2.1.2 RBF SVM

In [37]:
rbf_clf = SVC(kernel = 'rbf', gamma = 0.7, C=1, max_iter = 1000)    

rbf_clf.fit(X_train,y_train)
y_pred = rbf_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.9923611111111111

In [38]:
y_pred = rbf_clf.predict(X_test)
print(accuracy_score(y_test, y_pred))                   
print(confusion_matrix(y_test, y_pred) )

0.725
[[148   0   0  52]
 [ 30 124   1  45]
 [ 55   0 108  37]
 [  0   0   0 200]]


## 2.2 Logistic Regression

In [41]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

In [ ]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))                    
print(confusion_matrix(y_test, y_pred) )

## 2.3 KNN

In [39]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.9372222222222222

In [40]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))                    
print(confusion_matrix(y_test, y_pred) )

0.90125
[[160  14  18   8]
 [  3 191   1   5]
 [ 18   4 174   4]
 [  3   1   0 196]]


## 2.4 Gaussian Naive Bayes

In [33]:
from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.2526388888888889

In [22]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))                    
print(confusion_matrix(y_test, y_pred) )

0.30375
[[ 18   0  22 160]
 [  5   3   5 187]
 [  1   0  23 176]
 [  0   0   1 199]]


## 2.5 Decision Tree

In [23]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.9995833333333334

In [24]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))                    
print(confusion_matrix(y_test, y_pred) )

0.91875
[[164   7  22   7]
 [  3 197   0   0]
 [ 18   2 177   3]
 [  1   1   1 197]]


## 2.6 Random Forest

In [43]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_train)
accuracy_score(y_train, y_pred)

0.9965277777777778

In [26]:
y_pred = classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))                    
print(confusion_matrix(y_test, y_pred) )

0.9425
[[184   2  13   1]
 [  3 197   0   0]
 [ 18   4 178   0]
 [  4   1   0 195]]
